In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import scipy
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from dict_hash import sha256
import numpy as np
from torch.autograd import grad
import torch
from tqdm import tqdm
from torch.autograd.functional import jacobian
from torch.func import jacfwd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [35]:
class HDLM:
    def __init__(
            self, checkpoint,
            emb_size=10000,
            learning_rate=0.1,
            device='cuda',``
            seed=13,
    ):

        torch.manual_seed(seed)
        np.random.seed(seed)
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        vocab_size = len(self.tokenizer)
        self.learning_rate = learning_rate
        self.vocab = torch.rand((vocab_size, emb_size))  # uniform
        self.device = device
        self.seed = seed

    def next_emb_from_token_ids(self, input_ids):
        embs = self.get_embs(input_ids)
        return self.next_emb_from_embs(embs)

    def next_emb_from_embs(self, embs):
        '''Returns array of shape (emb_size)
        '''
        return torch.mean(embs, dim=0).squeeze()

    def get_embs(self, input_ids):
        '''Returns array of shape (len(input_ids), emb_size)
        '''
        if isinstance(input_ids, int) or isinstance(input_ids, float):
            input_ids = [input_ids]
        return torch.vstack([self.vocab[i] for i in input_ids])

    def emb_to_token_id(self, emb):
        '''Returns token id
        '''
        return torch.argmin(torch.norm(self.vocab - emb, dim=1))

    def update_vocab_emb(self, predicted_emb, next_token_correct_id):
        print(next_token_correct_id, predicted_emb.shape)
        emb = self.vocab[next_token_correct_id]
        self.vocab[next_token_correct_id] = (
            1 - self.learning_rate) * emb + self.learning_rate * predicted_emb

In [43]:
checkpoint = 'gpt2'
# checkpoint = 'bert-base-uncased'
tok = AutoTokenizer.from_pretrained(checkpoint)
start = 'Roses are red, violets are'
end = ' blue'
lm = HDLM(checkpoint, device='cpu', emb_size=10000, learning_rate=0.9)

In [44]:
print('tokens as strings', lm.tokenizer.tokenize(start))
input_ids = lm.tokenizer.encode(start, return_tensors='pt').to(lm.device)
print('tokens as ids', input_ids)

lm.get_embs(input_ids).shape

next_emb = lm.next_emb_from_token_ids(input_ids)
print('next_emb', next_emb.shape)

next_token_id = lm.emb_to_token_id(next_emb)
print('next_token', next_token_id, lm.tokenizer.decode(next_token_id))

target = lm.tokenizer(end)['input_ids']
lm.update_vocab_emb(next_emb, target[0])

next_token_id = lm.emb_to_token_id(next_emb)
print('next_token', next_token_id, lm.tokenizer.decode(next_token_id))

tokens as strings ['R', 'oses', 'Ġare', 'Ġred', ',', 'Ġv', 'io', 'lets', 'Ġare']
tokens as ids tensor([[  49, 4629,  389, 2266,   11,  410,  952, 5289,  389]])
